## Notebook to create Unity Catalog functions, Bedrock Agents and Action groups.

In [26]:
import logging
import sys

from unitycatalog.ai.bedrock.toolkit import UCFunctionToolkit
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [27]:
from unitycatalog.ai.bedrock.envs.bedrock_env_vars import BedrockEnvVars

bedrock_env = BedrockEnvVars.get_instance()

In [28]:
def setup_uc_client():
    """Set up Unity Catalog client"""
    config = Configuration()
    config.host = "http://0.0.0.0:8080/api/2.1/unity-catalog"
    api_client = ApiClient(configuration=config)
    return UnitycatalogFunctionClient(api_client=api_client)

In [29]:
client = setup_uc_client()

In [30]:
CATALOG = "AICatalog"
SCHEMA = "AISchema"

In [31]:
def get_weather_in_celsius(location_id: str, fetch_date: str) -> str:
    """
    Fetches weather data (in Celsius) for a given location and date.

    Args:
        location_id (str): A unique identifier for the location.
        fetch_date (str): The date to fetch the weather for (format: YYYY-MM-DD).

    Raises:
        Exception: For unexpected errors during execution.

    Returns:
        str: Temperature in Celsius.
    """
    try:
        return str(23)
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}") from e

In [32]:
def get_weather_in_fahrenheit(location_id: str, fetch_date: str) -> str:
    """
    Fetches weather data (in fahrenheit) for a given location and date.

    Args:
        location_id (str): A unique identifier for the location.
        fetch_date (str): The date to fetch the weather for (format: YYYY-MM-DD).

    Raises:
        Exception: For unexpected errors during execution.

    Returns:
        str: Temperature in fahrenheit.
    """
    try:
        return str(72)
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}") from e

In [ ]:
client.uc.create_catalog(name=CATALOG, comment="Catalog for AI functions")

In [ ]:
client.uc.create_schema(catalog_name=CATALOG, name=SCHEMA, comment="Schema for AI functions")

In [ ]:
client.create_python_function(
    func=get_weather_in_celsius, catalog=CATALOG, schema=SCHEMA, replace=True
)

In [ ]:
client.create_python_function(
    func=get_weather_in_fahrenheit, catalog=CATALOG, schema=SCHEMA, replace=True
)

In [ ]:
client.list_functions(catalog=CATALOG, schema=SCHEMA)

In [ ]:
function_name = f"{CATALOG}.{SCHEMA}.get_weather_in_celsius"
uc_f_toolkit = UCFunctionToolkit(function_names=[function_name], client=client)

logger.info(f"UC Function Toolkit:{uc_f_toolkit}")

### Bedrock Agents and Action groups.

In [39]:
import json
import logging
import sys
import time

import boto3

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)


iam_client = boto3.client("iam")
sts_client = boto3.client("sts")
session = boto3.session.Session()

In [ ]:
bedrock_env.aws_region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

logger.info(f"region:{bedrock_env.aws_region}")
logger.info(f"account_id:{account_id}")

In [ ]:
# Agent foundation model id
bedrock_env.bedrock_model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
# Agent foundation model Invoke requests per minute
# Check the model's service quota for the allowed limits
bedrock_env.bedrock_rpm_limit = 1

In [42]:
import uuid

unique_id = str(uuid.uuid4())[:8]

In [ ]:
# Create bedrock agent IAM policy
agent_bedrock_allow_policy_name = f"ucai-iam-policy-name-{unique_id}"

# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{bedrock_env.aws_region}::foundation-model/{bedrock_env.bedrock_model_id}"
            ],
        },
        {
            "Sid": "AmazonBedrockAgentStreamBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModelWithResponseStream",
            "Resource": [
                f"arn:aws:bedrock:{bedrock_env.aws_region}::foundation-model/{bedrock_env.bedrock_model_id}"
            ],
        },
    ],
}


bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name, PolicyDocument=bedrock_policy_json
)

logger.info(agent_bedrock_policy)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
agent_role_name = f"ucai-iam-role-name-{unique_id}"

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "bedrock.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name, AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name, PolicyArn=agent_bedrock_policy["Policy"]["Arn"]
)

In [ ]:
logger.info(agent_role)

In [ ]:
# Create bedrock agent

bedrock_env.bedrock_agent_name = f"ucai-bedrock-agent-{unique_id}"
agent_description = "ucai-bedrock-agent-description"
agent_instruction = "You are a weather agent to fetch the current weather in celsius and fahrenheit for a given location"

bedrock_agent_client = boto3.client("bedrock-agent")

create_agent_response = bedrock_agent_client.create_agent(
    agentName=bedrock_env.bedrock_agent_name,
    agentResourceRoleArn=agent_role["Role"]["Arn"],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=bedrock_env.bedrock_model_id,
    instruction=agent_instruction,
)
logger.info(create_agent_response)

In [ ]:
bedrock_env.bedrock_agent_id = create_agent_response["agent"]["agentId"]
logger.info(f"agent_id:{bedrock_env.bedrock_agent_id }")

In [48]:
# Prepare the function specifications
agent_functions = [
    {
        "name": "get_weather_in_celsius",
        "description": "Fetch the current weather in celsius for a given location and date",
        "parameters": {
            "location_id": {
                "description": "The unique identifier of the location to retrieve the temperature for",
                "required": True,
                "type": "string",
            },
            "fetch_date": {
                "description": "The specific date for which the temperature needs to be retrieved",
                "required": True,
                "type": "string",
            },
        },
        "requireConfirmation": "ENABLED",
    },
    {
        "name": "get_weather_in_fahrenheit",
        "description": "Fetch the current weather in fahrenheit for a given location and date",
        "parameters": {
            "location_id": {
                "description": "The unique identifier of the location to retrieve the temperature for",
                "required": True,
                "type": "string",
            },
            "fetch_date": {
                "description": "The specific date for which the temperature needs to be retrieved",
                "required": True,
                "type": "string",
            },
        },
        "requireConfirmation": "ENABLED",
    },
]

In [ ]:
# Prepare agent group using function schema
agent_action_group_name = f"ucai-bda-action-group-name-{unique_id}"
agent_action_group_description = "Actions to fetch the weather of a given location for a given date"

agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=bedrock_env.bedrock_agent_id,
    agentVersion="DRAFT",
    actionGroupExecutor={"customControl": "RETURN_CONTROL"},
    actionGroupName=agent_action_group_name,
    functionSchema={"functions": agent_functions},
    description=agent_action_group_description,
)
logger.info(agent_action_group_response)

In [ ]:
response = bedrock_agent_client.prepare_agent(agentId=bedrock_env.bedrock_agent_id)
logger.info(response)

In [ ]:
# Create bedrock agent alias
agent_alias_name = f"ucai-bedrock-agent-alias-{unique_id}"
agent_alias_description = "Alias for the weather agent"

create_alias_response = bedrock_agent_client.create_agent_alias(
    agentId=bedrock_env.bedrock_agent_id,
    agentAliasName=agent_alias_name,
    description=agent_alias_description
)

logger.info(create_alias_response)
bedrock_env.bedrock_agent_alias_id = create_alias_response['agentAlias']['agentAliasId']
logger.info(f"agent_alias_id:{bedrock_env.bedrock_agent_alias_id}")

In [ ]:
bedrock_env.save_to_file()